# Install pipelines SDK

In [ ]:
%env RELEASE_VERSION=0.1.18
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade

# Set appropriate sys path and import kfp pkgs

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [ ]:
client = kfp.Client()
client.list_experiments()

# Create a Dkube MNIST experiment

In [ ]:
mnist_experiment = client.create_experiment(name='Dkube - Mnist pl')

# Define e2e MNIST Pipeline with Dkube components

In [ ]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

dkube_training_op           = components.load_component_from_file("../components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("../components/serving/component.yaml")
dkube_viewer_op             = components.load_component_from_file('../components/viewer/component.yaml')

@dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist digits pipeline with dkube components'
)
def d3pipeline(
    access_url,
    training_container=json.dumps({'image':'docker.io/ocdr/dkube-datascience-tf-gpu:v1.12', 'username':'', 'password': ''}),
    training_program="mnist",
    training_script="python model.py",
    training_datasets=json.dumps(["mnist"]),
    training_gpus=1,
    training_envs=json.dumps([{"steps": 100}])):


    auth_token  = os.getenv("ACCESS_TOKEN")
    train       = dkube_training_op(auth_token, training_container,
                                    program=training_program, run_script=training_script,
                                    datasets=training_datasets, ngpus=training_gpus,
                                    envs=training_envs, access_url=access_url)
    serving     = dkube_serving_op(auth_token, train.outputs['artifact'], access_url=access_url).after(train)
    inference   = dkube_viewer_op(auth_token, serving.outputs['servingurl'],
                                  'digits', viewtype='inference', access_url=access_url).after(serving)

# Compile and generate tar ball

In [ ]:
compiler.Compiler().compile(d3pipeline, 'dkube_mnist_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

# Create and Run pipeline

#### Click the dkube-training stage to see the enhanced Dkube Datascience dashboard, metrics and graphs. Click the dkube-viewer stage for the simple UI to test the model predecitions

In [ ]:
run = client.run_pipeline(mnist_experiment.id, 'mnist_classifier_pipeline', 'dkube_mnist_pl.tar.gz', params={'access_url': "https://192.168.200.44:32222/"})